# Top-Level Consistency Checks

This notebook checks the top-level `tiles-SPECPROD.(fits|csv)` and `exposures-SPECPROD.(fits|csv)` files for self-consistency in prepartion for loading these files into a database.

## Imports

In [17]:
import os
import glob
from pytz import utc
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn
from astropy.time import Time
from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
from desispec.database.util import targetphotid
from desitarget.targets import decode_targetid
freeze_iers()

## Important Setup

In [2]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = True

In [3]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', 'edr', 'spectro', 'redux')

## Examine ztile files.

In [ ]:
all_targetphotid = list()
ztile_files = glob.glob(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-*.fits'))
for f in ztile_files:
    survey = os.path.basename(f).split('-')[1]
    if 'cumulative' in f:
        key = 'LASTNIGHT'
    elif 'perexp' in f:
        key = 'EXPID'
    elif 'pernight' in f:
        key = 'NIGHT'
    else:
        key = 'SPGRPVAL'
    t = Table.read(f, format='fits', hdu='ZCATALOG')
    objid, brickid, release, mock, sky, gaiadr = decode_targetid(t['TARGETID'])
    # print(sky)
    assert (t[key] == t['SPGRPVAL']).all()
    w = (t['TARGETID'] > 0) & (sky == 0)
    all_targetphotid += [targetphotid(x, y, survey) for x, y in zip(t['TARGETID'][w], t['TILEID'][w])]

## Test loading targeting data

In [4]:
release = 'edr'
targetphot_version = 'v1.0'
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'spectro', 'redux')
targetphot_potential_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'potential-targets', f"targetphot-potential-{os.environ['SPECPROD']}.fits")
targetphot_potential = Table.read(targetphot_potential_file, hdu=1)

KeyboardInterrupt: 

In [5]:
targetphot_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'observed-targets', f"targetphot-{os.environ['SPECPROD']}.fits")
targetphot = Table.read(targetphot_file, hdu=1)

### For a given `TARGETID`, which columns are always the same?

In [6]:
targetphot

RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ [5],EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET,SURVEY,PROGRAM,TILEID
int16,int32,bytes8,int32,bytes4,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32,float32,float32,float32,int16,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,float32,float32,float32,float32,bytes1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes7,bytes6,int32
9010,494512,0237p297,1707,DEV,23.661967677367254,80742950000.0,29.84758879289675,149970550000.0,8011.557 .. 9655.789,0.056008916,0.8742358,4.4879527,14.53286,846.09424,161.24467,27.071745,0.84721756,0.89432687,0.9394405,0.0010668915,0.86366457,0.0006714458,0.07150158,0.04451084,0.05153686,0.9731281,0.7351435,0.78252596,4,3,4,2643.3464,729.7028,87.75594,1827.4443,399.16907,56.263733,40.183647,23.470558,12.924044,230.56143,2.1023846,0.56624377,0.0014585183,2.1306869e-05,0.9905531,0.9941877,0.99875754,0.9995307,0,0,0,0.30432662,1.5622828,5.0589743,0.30432662,1.5622828,5.0589743,0.0,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,1.4857041,-0.47312373,0.34610084,964.16534,2238.5742,2086.7612,4.0,0.0,0,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,S,39628473198708395,0.3743222091683128,7,3410,2,2154,9007199254742016,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,cmx,other,80615
9010,506890,0225p322,3722,DEV,22.632277706310983,24347576000.0,32.147682585477945,25480208000.0,1907.2046 .. 2092.7915,0.04842441,0.33290792,1.8065038,7.54596,517.70844,140.78175,27.471045,0.8664541,0.9079553,0.94742143,0.014109313,0.0335639,0.008898258,0.026616279,0.027603306,0.029831763,0.666616,0.671309,0.9998857,2,2,2,1215.8024,403.50443,67.47949,816.19824,238.12308,43.442795,26.32007,9.095648,3.3897743,-641.0113,2.371093,0.6180614,0.0014815918,1.9939984e-05,0.9918271,0.9949728,0.9989257,0.9995942,0

In [10]:
ls_id = (targetphot['RELEASE'].astype(np.int64) << 40) | (targetphot['BRICKID'].astype(np.int64) << 16) | targetphot['BRICK_OBJID'].astype(np.int64)
ls_id.name = 'LS_ID'

In [11]:
ls_id

9906632174601899
9906632985808522
9906632985809029
9906632985809300
9906632985809304
9906632985809593
9906632985809681
9906632985809872
9906632985809900
9906632985870748
9906632985870940


In [13]:
unique_targetid = np.unique(targetphot['TARGETID'])
unique_targetid.name = 'UNIQUE_TARGETID'

In [39]:
for t in unique_targetid[:200]:
    rows = targetphot[targetphot['TARGETID'] == t]
    if len(rows) > 1:
        for c in ('RELEASE', 'BRICKID', 'BRICKNAME', 'BRICK_OBJID', 'MORPHTYPE',
                  'RA', 'RA_IVAR', 'DEC', 'DEC_IVAR',
                  'DCHISQ',
                  'EBV',
                  'FLUX_G', 'FLUX_R', 'FLUX_Z',
                  'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z',
                  'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z',
                  'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z',
                  'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z',
                  'FRACIN_G', 'FRACIN_R', 'FRACIN_Z',
                  'NOBS_G', 'NOBS_R', 'NOBS_Z',
                  'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z',
                  'GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z',
                  'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4',
                  'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4',
                  'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4',
                  'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z',
                  'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z',
                  'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z',
                  'REF_EPOCH',
                  'WISEMASK_W1', 'WISEMASK_W2',
                  'MASKBITS',
                  'LC_FLUX_W1', 'LC_FLUX_W2', 'LC_FLUX_IVAR_W1', 'LC_FLUX_IVAR_W2',
                  'LC_NOBS_W1', 'LC_NOBS_W1', 'LC_MJD_W1', 'LC_MJD_W2',
                  'SHAPE_R', 'SHAPE_E1', 'SHAPE_E2',
                  'SHAPE_R_IVAR', 'SHAPE_E1_IVAR', 'SHAPE_E2_IVAR',
                  'SERSIC', 'SERSIC_IVAR',
                  'REF_ID', 'REF_CAT',
                  'GAIA_PHOT_G_MEAN_MAG', 'GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_BP_MEAN_MAG', 'GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_RP_MEAN_MAG', 'GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_BP_RP_EXCESS_FACTOR', 'GAIA_ASTROMETRIC_EXCESS_NOISE',
                  'GAIA_DUPLICATED_SOURCE', 'GAIA_ASTROMETRIC_SIGMA5D_MAX', 'GAIA_ASTROMETRIC_PARAMS_SOLVED',
                  'PARALLAX', 'PARALLAX_IVAR', 'PMRA', 'PMRA_IVAR', 'PMDEC', 'PMDEC_IVAR',
                  'PHOTSYS', 'SUBPRIORITY', 'OBSCONDITIONS'):
            if isinstance(rows[c], MaskedColumn):
                try:
                    assert (rows[c].data.data == rows[c].data.data[0]).all()
                except:
                    print(t, rows[c].data)
            else:
                try:
                    assert (rows[c] == rows[c][0]).all()
                except:
                    print(t, rows[c])

In [43]:
for t in unique_targetid[:1000]:
    rows = targetphot[targetphot['TARGETID'] == t]
    if len(rows) > 1:
        for c in ('SUBPRIORITY', 'OBSCONDITIONS', 'PRIORITY_INIT', 'NUMOBS_INIT', 'HPXPIXEL',
                  'CMX_TARGET', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET',
                  'SV1_DESI_TARGET', 'SV1_BGS_TARGET', 'SV1_MWS_TARGET',
                  'SV2_DESI_TARGET', 'SV2_BGS_TARGET', 'SV2_MWS_TARGET',
                  'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET',
                  'SCND_TARGET', 'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET',
                  'SURVEY', 'PROGRAM', 'TILEID'):
            if isinstance(rows[c], MaskedColumn):
                try:
                    assert (rows[c].data.data == rows[c].data.data[0]).all()
                except:
                    print(t, rows[c].data)
            else:
                try:
                    assert (rows[c] == rows[c][0]).all()
                except:
                    print(t, rows[c])

6515536691200 TILEID
------
 80875
 80876
6521555517440 TILEID
------
 80889
 80890
6536638234624 TILEID
------
 80885
 80886


### Check for uniqueness.

In [ ]:
targetphot_potential_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot_potential['TARGETID'], targetphot_potential['TILEID'], targetphot_potential['SURVEY'])]

In [ ]:
targetphot_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot['TARGETID'], targetphot['TILEID'], targetphot['SURVEY'])]

In [ ]:
for t in targetphot_id:
    assert t in targetphot_potential_id

In [ ]:
for a in all_targetphotid:
    assert a in targetphot_id

### Do ztile files contain extraneous targets?

In [ ]:
ztile = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-special-dark-cumulative.fits'), hdu='ZCATALOG')

In [ ]:
w = ztile['TARGETID'] > 0

In [ ]:
np.unique(ztile['TILEID'])

In [ ]:
tid = [targetphotid(x, y, 'special') for x, y in zip(ztile['TARGETID'][w], ztile['TILEID'][w])]

In [ ]:
(158457821078535886096914448642 >> 64) & (2**32 - 1)

In [ ]:
(158457821078535886096914448642 >> 64) >> 32

In [ ]:
(158457821078535886096914448642) & (2**64 - 1)

In [ ]:
ztile[ztile['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TARGETID'] == 616088991480938754]

In [ ]:
targetphot_potential[targetphot_potential['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TILEID'] == 81101]

### Test load targetphot files.

In [ ]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
dsr.log = get_logger()
postgresql = dsr.setup_db(schema=specprod+'_target', overwrite=overwrite, hostname='nerscdb03.nersc.gov', verbose=True)

In [ ]:
loader = [{'filepaths': [os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-potential-targets-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-missing-{specprod}.fits'.format(specprod=os.environ['SPECPROD']))],
           'tcls': dsr.Target,
           'hdu': 'TARGETPHOT',
           'expand': {'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)},
           'q3c': 'ra',
           'chunksize': 100000,
           'maxrows': 0
           },]


In [ ]:
dsr.load_file(**(loader[0]))

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [ ]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [ ]:
tiles_fits

In [ ]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles_csv

### Are the two tiles files self-consistent?

In [ ]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(tiles_fits[row]['TILEID'], col, tiles_fits[row][col], tiles_csv[row][col])

### Load two versions of exposures file

In [ ]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures_fits

In [ ]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [ ]:
exposures_csv

### Are the two exposures files self-consistent?

In [ ]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(exposures_fits[row]['TILEID'], col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [ ]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

In [ ]:
np.unique(exposures_fits['PROGRAM'])

In [ ]:
np.unique(exposures_fits['GOALTYPE'])

In [ ]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

In [ ]:
np.unique(tiles_fits['PROGRAM'])

In [ ]:
assert (faflavor2program(tiles_fits['FAFLAVOR']) == tiles_fits['PROGRAM']).all

In [ ]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()

In [ ]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

### Compare frames to exposures

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [ ]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    try:
        assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    except AssertionError:
        print('EFFTIME_ETC', expid, exposures_fits[k]['TILEID'], exposures_fits[k]['EFFTIME_ETC'], frames[column][frames['EFFTIME_ETC'] == expid].tolist())
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    for column in ('MJD', 'EXPTIME', 'GOALTIME'):
        if column == 'GOALTIME' and (frames[column][frames['EXPID'] == expid] == 0).all():
            print(f"GOALTIME discrepancy for {expid}.")
        else:
            try:
                assert (np.around(frames[column][frames['EXPID'] == expid], 2) == np.around(exposures_fits[k][column], 2)).all()
            except AssertionError:
                try:
                    assert (np.around(frames[column][frames['EXPID'] == expid], 3) == np.around(exposures_fits[k][column], 3)).all()
                except AssertionError:
                    try:
                        assert (np.around(frames[column][frames['EXPID'] == expid], 4) == np.around(exposures_fits[k][column], 4)).all()
                    except AssertionError:
                        pass
                        # print(column, expid, exposures_fits[k][column], frames[column][frames['EXPID'] == expid].tolist())

### Compare tiles to exposures

In [ ]:
for row in tiles_fits:
    w = exposures_fits['TILEID'] == row['TILEID']
    assert len(exposures_fits[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures_fits[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures_fits[w][column])
    for column in ('TILERA', 'TILEDEC'):
        try:
            assert (np.around(exposures_fits[w][column], 2) == row[column]).all()
        except AssertionError:
            try:
                assert (np.around(exposures_fits[w][column], 3) == row[column]).all()
            except AssertionError:
                assert (np.around(exposures_fits[w][column], 4) == row[column]).all()
    for column in ('EXPTIME', 'EFFTIME_SPEC', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('EFFTIME_ETC', 'EFFTIME_GFA'):
        if (exposures_fits[w][column] == 0).any():
            assert row[column] == 0
        else:
            try:
                assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column], 1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.around(exposures_fits[w][column], 2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures_fits[w][column], decimals=1))
    
    assert exposures_fits[w]['NIGHT'].max() == row['LASTNIGHT']